Problem 1: Decision Tree for Classification (with Gini Index)
Problem Statement
You are tasked with implementing a decision tree classifier from scratch using NumPy, using the Gini index as the splitting criterion. The decision tree will classify synthetic 2D data points into two classes based on feature thresholds. The implementation will include tree construction, prediction, and evaluation using accuracy.
Mathematical Definition:

Gini Index for a node:
$$\text{Gini} = 1 - \sum_{i=1}^c p_i^2$$
where $ p_i $ is the proportion of class $ i $ in the node, and $ c $ is the number of classes.
Split Criterion: Choose the feature and threshold that minimize the weighted Gini index of child nodes:
$$\text{Gini}_{\text{split}} = \frac{n_{\text{left}}}{n} \text{Gini}_{\text{left}} + \frac{n_{\text{right}}}{n} \text{Gini}_{\text{right}}$$

Prediction: Assign the majority class of the leaf node.

Requirements

Implement a DecisionTreeClassifier class with methods for:

fit: Build the tree using recursive splitting.
predict: Classify new data points.


Use the Gini index to select splits.
Handle binary classification with 2D synthetic data.
Evaluate accuracy on a test set.

Constraints

Use only NumPy for data manipulation and tree logic.
No scikit-learn or other ML libraries.
Max tree depth of 3 to prevent overfitting.
Handle batch inputs for prediction.

In [ ]:
import numpy as np 
np.random.seed(42)

X=np.random.rand(100,2)*10
# Purpose: Create 100 data points with 2 features, values between 0 and 10.
# Theory: np.random.rand generates uniform random numbers in [0, 1). Scaling by 10 gives features in [0, 10). Shape [100, 2] represents 100 samples with 2 features.
y=(X[:,0]+X[:,1]>10).astype(int)
# Purpose: Create binary labels based on a condition. If the sum of features is greater than 10, label is 1, else 0.
# Theory: X[:,0] and X[:,1] access the first and second feature columns. The condition checks if their sum exceeds 10, converting the boolean result to integers (0 or 1).

#define the Decision Tree classifier 
class DecisionTreeClassifier: